<a href="https://colab.research.google.com/github/abdulmajee/news_data_augmetation/blob/main/3009_doc_Fine_tuning_BERT_(dbmdz_bert_base_turkish_cased)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [1]:
!pip install -q transformers datasets

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import datasets
from datasets import load_dataset, DatasetDict


# Load original daasets from google drive
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Colab Notebooks/metin_dosyasi_dataframe.csv', split = 'train')

Using custom data configuration default-1b738122277497e5
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-1b738122277497e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [4]:
dataset

Dataset({
    features: ['text', 'Careers', 'Personal_Finance', 'Religion_&_Spirituality', 'Travel', 'Science', 'Food_&_Drink', 'Events_and_Attractions', 'Real_Estate', 'Family_and_Relationships', 'Sports', 'Healthy_Living', 'Shopping', 'News_and_Politics', 'Television', 'Movies', 'Pop_Culture', 'Hobbies_&_Interests', 'Home_&_Garden', 'Pets', 'Automotive', 'Medical_Health', 'Books_and_Literature', 'Style_&_Fashion', 'Fine_Art', 'Technology_&_Computing', 'Music_and_Audio', 'Business_and_Finance', 'Video_Gaming', 'Education'],
    num_rows: 3009
})

In [5]:
# dataset = dataset.remove_columns("bilgilHealthy_Living")

In [6]:
metin_train_devtest = dataset.train_test_split(shuffle = True, seed = 200, test_size=0.1)
metin_dev_test = metin_train_devtest['test'].train_test_split(shuffle = True, seed = 200, test_size=0.50)
dataset = DatasetDict({
    'train': metin_train_devtest['train'],
    'test': metin_dev_test['test'],
    'dev': metin_dev_test['train']})

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-1b738122277497e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-3540db54bb0d9bc0.arrow and /root/.cache/huggingface/datasets/csv/default-1b738122277497e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-6375d2882b0ad4bd.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-1b738122277497e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-70e4598fcddf3b19.arrow and /root/.cache/huggingface/datasets/csv/default-1b738122277497e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-ca8ef2e4bdd92296.arrow


Let's check the first example of the training split:

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'Careers', 'Personal_Finance', 'Religion_&_Spirituality', 'Travel', 'Science', 'Food_&_Drink', 'Events_and_Attractions', 'Real_Estate', 'Family_and_Relationships', 'Sports', 'Healthy_Living', 'Shopping', 'News_and_Politics', 'Television', 'Movies', 'Pop_Culture', 'Hobbies_&_Interests', 'Home_&_Garden', 'Pets', 'Automotive', 'Medical_Health', 'Books_and_Literature', 'Style_&_Fashion', 'Fine_Art', 'Technology_&_Computing', 'Music_and_Audio', 'Business_and_Finance', 'Video_Gaming', 'Education'],
        num_rows: 2708
    })
    test: Dataset({
        features: ['text', 'Careers', 'Personal_Finance', 'Religion_&_Spirituality', 'Travel', 'Science', 'Food_&_Drink', 'Events_and_Attractions', 'Real_Estate', 'Family_and_Relationships', 'Sports', 'Healthy_Living', 'Shopping', 'News_and_Politics', 'Television', 'Movies', 'Pop_Culture', 'Hobbies_&_Interests', 'Home_&_Garden', 'Pets', 'Automotive', 'Medical_Health', 'Books_and_Literatu

In [8]:
example = dataset['train'][0]
example

{'Automotive': 0,
 'Books_and_Literature': 0,
 'Business_and_Finance': 0,
 'Careers': 0,
 'Education': 0,
 'Events_and_Attractions': 0,
 'Family_and_Relationships': 0,
 'Fine_Art': 0,
 'Food_&_Drink': 0,
 'Healthy_Living': 0,
 'Hobbies_&_Interests': 0,
 'Home_&_Garden': 0,
 'Medical_Health': 0,
 'Movies': 0,
 'Music_and_Audio': 0,
 'News_and_Politics': 0,
 'Personal_Finance': 0,
 'Pets': 0,
 'Pop_Culture': 0,
 'Real_Estate': 1,
 'Religion_&_Spirituality': 0,
 'Science': 0,
 'Shopping': 0,
 'Sports': 0,
 'Style_&_Fashion': 0,
 'Technology_&_Computing': 0,
 'Television': 0,
 'Travel': 0,
 'Video_Gaming': 0,
 'text': 'portville portville kiralık villa portville göcek i̇çeriğe atla holiday villas turkey vacation rentals vacation lettings gocek fethiye turkey menü kiralık göcek kiralık evler gökçeovacık kiralık evler i̇nlice kiralık evler sarıgerme kiralık evler sultaniye kiralık evler kiralık daireler kiralık doğa evleri kiralık villalar portville portville göcek kiralık portville göcek sa

The dataset consists of text documents, labeled with one or more categories. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [9]:
labels = [label for label in dataset['train'].features.keys() if label not in ['text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Careers',
 'Personal_Finance',
 'Religion_&_Spirituality',
 'Travel',
 'Science',
 'Food_&_Drink',
 'Events_and_Attractions',
 'Real_Estate',
 'Family_and_Relationships',
 'Sports',
 'Healthy_Living',
 'Shopping',
 'News_and_Politics',
 'Television',
 'Movies',
 'Pop_Culture',
 'Hobbies_&_Interests',
 'Home_&_Garden',
 'Pets',
 'Automotive',
 'Medical_Health',
 'Books_and_Literature',
 'Style_&_Fashion',
 'Fine_Art',
 'Technology_&_Computing',
 'Music_and_Audio',
 'Business_and_Finance',
 'Video_Gaming',
 'Education']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [10]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245k [00:00<?, ?B/s]

In [11]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Parameter 'function'=<function preprocess_data at 0x7f2383c894d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [13]:
tokenizer.decode(example['input_ids'])

'[CLS] portville portville kiralık villa portville göcek i̇çeriğe atla holiday villas turkey vacation rentals vacation lettings gocek fethiye turkey menü kiralık göcek kiralık evler gökçeovacık kiralık evler i̇nlice kiralık evler sarıgerme kiralık evler sultaniye kiralık evler kiralık daireler kiralık doğa evleri kiralık villalar portville portville göcek kiralık portville göcek satılık satılık villa daire doğa evi arsa arazi ticari tekne kiralık tekne kiralık lobster tekne reggae kiralık yelkenli bavaria 36 kiralık motoryat rınker 27 kiralık motoryat imera [SEP]'

In [14]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [15]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Real_Estate']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [16]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [17]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [18]:
batch_size = 8
metric_name = "f1"

In [20]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-base-turkish-cased-finetuned-3009metin-dosya-turkish",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [21]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [22]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [23]:
encoded_dataset['train']['input_ids'][0]

tensor([    2,  8589, 24967,  8589, 24967,  9256, 16866,  8589, 24967,  2036,
         2966,    77,  1305, 19777, 10338,  2582,  1971, 18559,  8465,  1032,
        16866,  1022, 14335,  2018,  8263, 24156, 14495, 16092,  2008,  1022,
         8263, 24156, 14495,  6181, 26080, 19643,  9153,  2966, 15116,  2092,
        14335,  2018, 13916,  9256,  2036,  2966,  9256, 16303,  7052,  2581,
         4115,  5794,  9256, 16303,    77,  1305,  1009, 17285,  1025,  9256,
        16303,  6657,  5134,  2078,  9256, 16303, 17387,  2092,  9256, 16303,
         9256, 23276,  9256,  9691, 16130,  9256, 16866,  1980,  8589, 24967,
         8589, 24967,  2036,  2966,  9256,  8589, 24967,  2036,  2966, 11544,
        11544, 16866,  9420,  9691,  8536, 13332, 12353,  6675, 16245,  9256,
        16245,  9256, 20702,  2405, 16245, 18880,  3624,  1025,  9256, 18349,
         2031,  3862, 26890,  1006,  5577,  9256,  4866, 11060,    86,  1976,
         6933,  4377,  9256,  4866, 11060,  2697,  5004,     3])

In [24]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput([('loss',
                           tensor(0.7253, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)),
                          ('logits',
                           tensor([[-0.1711,  0.1353, -0.3556, -0.3489, -0.5810,  0.3280,  0.7590,  0.8785,
                                     0.1871,  0.1319, -0.3137, -0.2073, -0.0723,  0.1970,  0.1933, -0.1610,
                                     0.1579,  0.5345, -0.1100,  0.1389, -0.8697,  0.8899,  0.0530, -0.6766,
                                     0.2165,  0.3625,  0.6660,  0.0812,  0.2483]],
                                  grad_fn=<AddmmBackward0>))])

Let's start training!

In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2708
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1695


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.157244,0.073059,0.518788,0.000000
2,0.192100,0.104241,0.519031,0.681841,0.293333
3,0.095600,0.084933,0.722388,0.794036,0.526667
4,0.095600,0.075452,0.782857,0.833181,0.600000
5,0.066700,0.072883,0.780627,0.833061,0.593333


***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to bert-base-turkish-cased-finetuned-3009metin-dosya-turkish/checkpoint-339
Configuration saved in bert-base-turkish-cased-finetuned-3009metin-dosya-turkish/checkpoint-339/config.json
Model weights saved in bert-base-turkish-cased-finetuned-3009metin-dosya-turkish/checkpoint-339/pytorch_model.bin
tokenizer config file saved in bert-base-turkish-cased-finetuned-3009metin-dosya-turkish/checkpoint-339/tokenizer_config.json
Special tokens file saved in bert-base-turkish-cased-finetuned-3009metin-dosya-turkish/checkpoint-339/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to bert-base-turkish-cased-finetuned-3009metin-dosya-turkish/checkpoint-678
Configuration saved in bert-base-turkish-cased-finetuned-3009metin-dosya-turkish/checkpoint-678/config.json
Model weights saved in bert-base-turkish-cased-finetuned-3009metin-dosya-turkish

TrainOutput(global_step=1695, training_loss=0.11134828663153634, metrics={'train_runtime': 383.9382, 'train_samples_per_second': 35.266, 'train_steps_per_second': 4.415, 'total_flos': 890846831016960.0, 'train_loss': 0.11134828663153634, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [27]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 150
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.6,
 'eval_f1': 0.7828571428571429,
 'eval_loss': 0.07545159012079239,
 'eval_roc_auc': 0.8331813233693255,
 'eval_runtime': 1.2345,
 'eval_samples_per_second': 121.508,
 'eval_steps_per_second': 15.391}

## Inference

Let's test the model on a new sentence:

In [31]:
text = "Koca, Twitter hesabından yaptığı paylaşımda,Şoför arkadaşımız arzusu üzerine misafirim oldu. Hasta nakil aracıyla naklini yaptığı hastanın Ankara Şehir Hastanesi aciline yatışını sağladıktan 2,5 saat sonra aracıyla bakanlık binası önüne park edişinden başladık, çok şey konuştuk. Ama bazı şeyler konuşulduğu yerde kalmalı"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [32]:
logits = outputs.logits
logits.shape

torch.Size([1, 29])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [33]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

[]
